In [2]:

# First import the library
import pyrealsense2 as rs
import time
import json

In [2]:

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 60)
config.enable_stream(rs.stream.color, 960, 540, rs.format.bgr8, 60)

pipeline.start(config)
# Create an align object
# rs.align allows us to perform alignment of depth frames to others frames
# The "align_to" is the stream type to which we plan to align depth frames.
align_to = rs.stream.color
align = rs.align(align_to)

ctx = rs.context()
devices = ctx.query_devices()
for dev in devices:
    if dev.supports(rs.camera_info.product_id) and dev.supports(rs.camera_info.name):
        print("Found camera device: {}".format(dev.get_info(rs.camera_info.name)))
        sensors = dev.query_sensors()
        for sensor in sensors:
            if sensor.get_info(rs.camera_info.name) == "RGB Camera" and sensor.supports(
                    rs.option.exposure) and sensor.supports(rs.option.gain):
                print("Setting RGB camera sensor settings")
                sensor.set_option(rs.option.enable_auto_exposure, 1)
                sensor.set_option(rs.option.white_balance, 3000)
                sensor.set_option(rs.option.enable_auto_white_balance, 0)
                print("exposure: {}".format(sensor.get_option(rs.option.exposure)))
                print("white balance: {}".format(sensor.get_option(rs.option.white_balance)))
                print("gain: {}".format(sensor.get_option(rs.option.gain)))
                time.sleep(2)
                sensor.set_option(rs.option.enable_auto_exposure, 0)
                print(
                    "auto exposure enabled: {}".format(sensor.get_option(rs.option.enable_auto_exposure)))
                print("exposure: {}".format(sensor.get_option(rs.option.exposure)))  # 166
                print("white balance: {}".format(sensor.get_option(rs.option.white_balance)))
                print("gain: {}".format(sensor.get_option(rs.option.gain)))  # 64
                break
        break

def get_frame(self):
    frames = self.pipeline.wait_for_frames()
    aligned_frames = self.align.process(frames)
    aligned_depth_frame = aligned_frames.get_depth_frame()  # aligned_depth_frame is a 640x480 depth image
    aligned_color_frame = aligned_frames.get_color_frame()
    if not aligned_depth_frame or not aligned_color_frame:
        return

    self.depth_image = np.asanyarray(aligned_depth_frame.get_data())
    self.regular_image = np.asanyarray(aligned_color_frame.get_data())
    # self.yuv = cv2.cvtColor(self.regular_image, cv2.COLOR_BGR2YUV)
    self.hsv = cv2.cvtColor(self.regular_image, cv2.COLOR_BGR2HSV)



Found camera device: Intel RealSense D435
Setting RGB camera sensor settings
exposure: 166.0
white balance: 3000.0
gain: 64.0
auto exposure enabled: 0.0
exposure: 166.0
white balance: 3000.0
gain: 64.0


In [5]:
one()

Found camera device: Intel RealSense D435
Setting RGB camera sensor settings
exposure: 166.0
white balance: 3000.0
gain: 64.0
auto exposure enabled: 0.0
exposure: 166.0
white balance: 3000.0
gain: 64.0


In [4]:
DS5_product_ids = ["0AD1", "0AD2", "0AD3", "0AD4", "0AD5", "0AF6", "0AFE", "0AFF", "0B00", "0B01", "0B03", "0B07"]

def find_device_that_supports_advanced_mode() :
    ctx = rs.context()
    ds5_dev = rs.device()
    devices = ctx.query_devices();
    for dev in devices:
        if dev.supports(rs.camera_info.product_id) and str(dev.get_info(rs.camera_info.product_id)) in DS5_product_ids:
            if dev.supports(rs.camera_info.name):
                print("Found device that supports advanced mode:", dev.get_info(rs.camera_info.name))
            return dev
    raise Exception("No device that supports advanced mode was found")

In [5]:
try:
    dev = find_device_that_supports_advanced_mode()
    advnc_mode = rs.rs400_advanced_mode(dev)
    print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Loop until we successfully enable advanced mode
    while not advnc_mode.is_enabled():
        print("Trying to enable advanced mode...")
        advnc_mode.toggle_advanced_mode(True)
        # At this point the device will disconnect and re-connect.
        print("Sleeping for 5 seconds...")
        time.sleep(5)
        # The 'dev' object will become invalid and we need to initialize it again
        dev = find_device_that_supports_advanced_mode()
        advnc_mode = rs.rs400_advanced_mode(dev)
        print("Advanced mode is", "enabled" if advnc_mode.is_enabled() else "disabled")

    # Serialize all controls to a Json string
    serialized_string = advnc_mode.serialize_json()
    as_json_object = json.loads(serialized_string)

    # We can also load controls from a json string
    # For Python 2, the values in 'as_json_object' dict need to be converted from unicode object to utf-8
    if type(next(iter(as_json_object))) != str:
        as_json_object = {k.encode('utf-8'): v.encode("utf-8") for k, v in as_json_object.items()}

    #Set auto white balance to False
    as_json_object['controls-color-white-balance-auto'] = 'False'
    # The C++ JSON parser requires double-quotes for the json object so we need
    # to replace the single quote of the pythonic json to double-quotes
    json_string = str(as_json_object).replace("'", '\"')
    advnc_mode.load_json(json_string)

except Exception as e:
    print(e)
pass

('Found device that supports advanced mode:', u'Intel RealSense D435')
('Advanced mode is', 'enabled')


In [7]:
as_json_object['controls-color-gain']

'64'

In [4]:
def on_notification(n):
    print (n)
    if n.category == rs.notification_category.hardware_event:
        try:
            event = json.loads(n.serialized_data)
            if event and event['Event Type'] == 'Controller Event' \
                and event['Data']['Sub Type'] == 'Discovery':
                addr = event['Data']['Data']['MAC']
                print ('Connecting to mac_str...')
                try:
                    tm2.connect_controller(addr)  # expecting tm2 = The device as tm2
                except:
                    print ('Failed to connect to controller ', mac_str)
        except:
            print ('Serialized data is not in JSON format (', \
                   n.serialized_data, ')')
            
def on_frame(f):
    pass

In [12]:
advnc_mode.toggle_advanced_mode(False)

In [20]:
rs.pipeline(rs.context())

In [14]:
c = dev.sensors[0]

In [15]:
c.start(on_frame)

RuntimeError: start_streaming(...) failed. UVC device was not opened!

In [4]:
 print("Controls as JSON: \n", serialized_string)

('Controls as JSON: \n', u'{\n    "aux-param-autoexposure-setpoint": "1536",\n    "aux-param-colorcorrection1": "0.298828",\n    "aux-param-colorcorrection10": "0",\n    "aux-param-colorcorrection11": "0",\n    "aux-param-colorcorrection12": "0",\n    "aux-param-colorcorrection2": "0.293945",\n    "aux-param-colorcorrection3": "0.293945",\n    "aux-param-colorcorrection4": "0.114258",\n    "aux-param-colorcorrection5": "0",\n    "aux-param-colorcorrection6": "0",\n    "aux-param-colorcorrection7": "0",\n    "aux-param-colorcorrection8": "0",\n    "aux-param-colorcorrection9": "0",\n    "aux-param-depthclampmax": "65536",\n    "aux-param-depthclampmin": "0",\n    "aux-param-disparityshift": "0",\n    "controls-autoexposure-auto": "True",\n    "controls-autoexposure-manual": "8500",\n    "controls-color-autoexposure-auto": "True",\n    "controls-color-autoexposure-manual": "166",\n    "controls-color-backlight-compensation": "0",\n    "controls-color-brightness": "0",\n    "controls-colo